In [1]:
import h5py
from model_xray.config import *
from model_xray.path_manager import pm

with h5py.File(pm.get_mcwa_path('famous_le_10m'), 'r') as f:
    for mz_name in f:
        print(mz_name)
        mz = f[mz_name]
        print(mz.dtype, mz.shape)

DenseNet121
float32 (1, 8062504)
EfficientNetV2B0
float32 (1, 7200312)
EfficientNetV2B1
float32 (1, 8212124)
MobileNet
float32 (1, 4253864)
MobileNetV2
float32 (1, 3538984)
MobileNetV3Large
float32 (1, 5507432)
MobileNetV3Small
float32 (1, 2554968)
NASNetMobile
float32 (1, 5326716)


In [1]:
from model_xray.config_classes import XLSBAttackConfig, PayloadType
from model_xray.utils.mal_embedding_utils import x_lsb_attack

import numpy as np

arr = np.zeros((10, ), dtype=np.uint8)

payload = np.array([0, 1, 2, 3, 4, 5, 6, 7, 20, 14], dtype=np.uint8)

config = XLSBAttackConfig(
    x = 8,
    fill=True,
    payload_type=PayloadType.PYTHON_BYTES,
    payload_bytes=payload.tobytes()
)

arr_attacked = x_lsb_attack(arr, config)
print(arr)
print(arr_attacked)

capacity: 80, n_b: 8
payload bits: 00000000000000010000001000000011000001000000010100000110000001110001010000001110, len(bits): 80
dupe_amount: 1
len(bits): 80
reshaped: Array('bin8', ['00000000', '00000001', '00000010', '00000011', '00000100', '00000101', '00000110', '00000111', '00010100', '00001110']), remainder: 
[0 0 0 0 0 0 0 0 0 0]
[ 0  1  2  3  4  5  6  7 20 14]
